In [ ]:
# Para el manejo eficiente del conjunto de datos
import pandas as pd 
import numpy as np
# Para el manejo de modelos estadísticos
from scipy import stats
# Para construir los elementos gráficos
import seaborn as sns 
import matplotlib.pyplot as plt
# Para la visualización de los valores nulos
import missingno as msno
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

import os



In [65]:
natalidad_2017 = pd.read_csv('./datos/sinac2017DatosAbiertos.csv',low_memory=False)
natalidad_2018 = pd.read_csv('./datos/sinac2018DatosAbiertos.csv',low_memory=False)
natalidad_2019 = pd.read_csv('./datos/sinac2019DatosAbiertos.csv',low_memory=False)




In [66]:
natalidad_2017_sonora=natalidad_2017[natalidad_2017['ENT_NAC']==26]
natalidad_2018_sonora=natalidad_2018[natalidad_2018['ENT_NAC']==26]
natalidad_2019_sonora=natalidad_2019[natalidad_2019['ENT_NAC']==26]


In [67]:

natalidad_2017_sonora=natalidad_2017_sonora.drop('Unnamed: 0',axis=1)
print(natalidad_2017_sonora.columns)

natalidad_2018_sonora=natalidad_2018_sonora.drop(labels='Unnamed: 0',axis=1)
print(natalidad_2018_sonora.columns)

print(natalidad_2019_sonora.columns)

Index(['CEDOCVE', 'ENT_NACM', 'MPO_NACM', 'FECH_NACM', 'EDADM', 'CON_INDM',
       'HABLA_INDM', 'CUAL_LENGM', 'EDOCIVIL', 'TIPOVIAL_RES', 'TIPOASEN_RES',
       'ENT_RES', 'MPO_RES', 'LOC_RES', 'NUM_EMB', 'NUM_NACMTO', 'NUM_NACVIVO',
       'HIJO_SOBV', 'HIJO_ANTE', 'VIVE_AUN', 'ORDEN_NAC', 'ATEN_PREN',
       'TRIM_CONS', 'TOT_CONS', 'SOB_PARTO', 'DERHAB', 'DERHAB2', 'NIV_ESCOL',
       'OCUPHAB', 'CVEOCUPHAB', 'TRAB_ACT', 'FECH_NACH', 'HORA_NACH', 'SEXOH',
       'GESTACH', 'TALLAH', 'PESOH', 'APGARH', 'SILVERMAN', 'BCG', 'HEP_B',
       'VIT_A', 'VIT_K', 'TAM_AUD', 'PRODUCTO', 'ACELRN', 'CVE_CIE', 'ACELRN2',
       'CVE_CIE2', 'PROCNAC', 'FORCEPS', 'ESPECIFIQUE', 'INST_NAC', 'UNIMED',
       'CLUES', 'ATENDIO', 'ATEN_OTRO', 'TIPOVIAL_NAC', 'TIPOASEN_NAC',
       'ENT_NAC', 'MPO_NAC', 'LOC_NAC', 'CERT_POR', 'OTROMEDICO',
       'UNIMED_33_1', 'CLUES_33_2', 'TIPOVIAL_CERT', 'TIPOASEN_CERT',
       'ENT_CERT', 'MPO_CERT', 'LOC_CERT', 'FECH_CERT', 'FECH_ALTA',
       'FECH_CAMB', 'IDCA

In [68]:
natalidadtotal=natalidad_2017_sonora.append(natalidad_2018_sonora)

natalidadtotal=natalidadtotal.append(natalidad_2019_sonora)

natalidadtotal.shape

(126867, 76)

In [83]:
natalidadtotal=natalidadtotal.loc[:,['ENT_NACM','MPO_NACM','FECH_NACM','EDADM','CON_INDM','HABLA_INDM','EDOCIVIL','ENT_RES','MPO_RES','LOC_RES','NUM_EMB','NUM_NACMTO','NUM_NACVIVO','HIJO_SOBV','HIJO_ANTE','VIVE_AUN','ORDEN_NAC','ATEN_PREN','TRIM_CONS','TOT_CONS','SOB_PARTO','DERHAB','NIV_ESCOL','OCUPHAB','TRAB_ACT','FECH_NACH','HORA_NACH','SEXOH','GESTACH','TALLAH','PESOH','APGARH','SILVERMAN','BCG','HEP_B','VIT_A','VIT_K','TAM_AUD','PRODUCTO','PROCNAC','FORCEPS','INST_NAC','CLUES','ATENDIO','ENT_NAC','MPO_NAC','LOC_NAC','CERT_POR','ENT_CERT','MPO_CERT','LOC_CERT','FECH_CERT']]

In [84]:
nuevas_columnas = {'ENT_NACM':'ENTIDADNACIMIENTO','MPO_NACM':'MUNICIPIONACIMIENTO','FECH_NACM':'FECHANACIMIENTOMADRE','EDADM':'EDAD','CON_INDM':'SECONSIDERAINDIGENA','HABLA_INDM':'HABLALENGUAINDIGENA','EDOCIVIL':'ESTADOCONYUGAL','ENT_RES':'ENTIDADRESIDENCIA','MPO_RES':'MUNICIPIORESIDENCIA','LOC_RES':'LOCALIDADRESIDENCIA','NUM_EMB':'NUMEROEMBARAZOS','NUM_NACMTO':'HIJOSNACIDOSMUERTOS','NUM_NACVIVO':'HIJOSNACIDOSVIVOS','HIJO_SOBV':'HIJOSSOBREVIVIENTES','HIJO_ANTE':'CONDICIONHIJOANTERIOR','VIVE_AUN':'VIVEHIJOANTERIOR','ORDEN_NAC':'ORDENNACIMIENTO','ATEN_PREN':'ATENCIONPRENATAL','TRIM_CONS':'TRIMESTREPRIMERCONSULTA','TOT_CONS':'TOTALCONSULTAS','SOB_PARTO':'SOBREVIVIOPARTO','DERHAB':'AFILIACION','NIV_ESCOL':'ESCOLARIDAD','OCUPHAB':'Ocupacion_Habitual','TRAB_ACT':'TRABAJAACTUALMENTE','FECH_NACH':'FECHANACIMIENTO','HORA_NACH':'HORANACIMIENTO','SEXOH':'SEXO','GESTACH':'EDADGESTACIONAL','TALLAH':'TALLA','PESOH':'PESO','APGARH':'APGAR','SILVERMAN':'SILVERMAN','BCG':'VACUNA_BCG','HEP_B':'VACUNAHEPATITIS_B','VIT_A':'VITAMINA_A','VIT_K':'VITAMINA_K','TAM_AUD':'TAMIZAUDITIVO','PRODUCTO':'PRODUCTOEMBARAZO','PROCNAC':'TIPOCESAREA','FORCEPS':'UTILIZOFORCEPS','INST_NAC':'LUGARNACIMIENTO','CLUES':'CLUES','ATENDIO':'PERSONALATENDIO','ENT_NAC':'ENTIDADFEDERATIVAPARTO','MPO_NAC':'MUNICIPIOPARTO','LOC_NAC':'LOCALIDADPARTO','CERT_POR':'CERTIFICADOPOR','ENT_CERT':'ENTIDADFEDERATIVACERTIFICA','MPO_CERT':'MUNICIPIOCERTIFICA','LOC_CERT':'LOCALIDADCERTIFICA','FECH_CERT':'FECHACERTIFICADO'}


In [90]:
natalidadtotal.rename(columns=nuevas_columnas,inplace=True)

In [111]:
natalidadtotal.columns
natalidadtotal.to_csv('./datos/Sonora2017_18_19.csv')

In [99]:
natalidad_2020 = pd.read_csv('./datos/sinac2020DatosAbiertos.csv',low_memory=False)

In [100]:
natalidad_2020_sonora=natalidad_2020[natalidad_2020['ENTIDADFEDERATIVAPARTO']==26]

In [104]:
natalidad_2020_sonora

,NACIOEXTRANJERO,ENTIDADNACIMIENTO,MUNICIPIONACIMIENTO,EDAD,SECONSIDERAINDIGENA,HABLALENGUAINDIGENA,FECHANACIMIENTOMADRE,ESTADOCONYUGAL,RESIDEEXTRANJERO,ENTIDADRESIDENCIA,...,TIPOMEDICOATENDIO,ENTIDADFEDERATIVAPARTO,MUNICIPIOPARTO,LOCALIDADPARTO,CERTIFICADOPOR,CLUESCERTIFICA,ENTIDADFEDERATIVACERTIFICA,MUNICIPIOCERTIFICA,LOCALIDADCERTIFICA,FECHACERTIFICADO
24,2,26,30,25,2,2,14/11/1994,5,2,26,...,11.0,26,30,1,1,NaN,26,30,1,03/01/2020
31,2,26,18,32,2,2,31/03/1987,4,2,26,...,11.0,26,30,1,1,NaN,26,30,1,15/01/2020
88,2,25,17,39,2,2,29/01/1980,5,2,26,...,11.0,26,55,1,7,NaN,26,55,1,15/01/2020
91,1,0,999,16,2,2,27/10/2003,4,2,26,...,11.0,26,43,1,2,NaN,26,43,1,10/01/2020
341,2,26,36,24,2,2,12/05/1995,5,2,26,...,11.0,26,36,1,2,NaN,26,36,1,27/01/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723539,2,17,7,25,2,2,14/04/1995,9,2,26,...,11.0,26,17,1,7,NaN,26,17,1,22/08/2020
1723540,2,17,7,28,2,2,10/01/1992,5,2,26,...,12.0,26,17,1,7,NaN,26,17,1,07/06/2020
1723541,2,10,2,40,2,2,12/06/1979,5,2,26,...,12.0,26,17,1,7,NaN,26,17,1,26/01/2020
1723542,2,26,17,36,2,2,12/02/1984,5,2,26,...,12.0,26,17,1,7,NaN,26,17,1,16/06/2020


In [108]:
natalidad_2020_sonora=natalidad_2020_sonora.loc[:,['ENTIDADNACIMIENTO','MUNICIPIONACIMIENTO','FECHANACIMIENTOMADRE','EDAD','SECONSIDERAINDIGENA','HABLALENGUAINDIGENA','ESTADOCONYUGAL','ENTIDADRESIDENCIA','MUNICIPIORESIDENCIA','LOCALIDADRESIDENCIA','NUMEROEMBARAZOS','HIJOSNACIDOSMUERTOS','HIJOSNACIDOSVIVOS','HIJOSSOBREVIVIENTES','CONDICIONHIJOANTERIOR','VIVEHIJOANTERIOR','ORDENNACIMIENTO','ATENCIONPRENATAL','TRIMESTREPRIMERCONSULTA','TOTALCONSULTAS','SOBREVIVIOPARTO','AFILIACION','ESCOLARIDAD','TRABAJAACTUALMENTE','FECHANACIMIENTO','HORANACIMIENTO','SEXO','EDADGESTACIONAL','TALLA','PESO','APGAR','SILVERMAN','VACUNA_BCG','VACUNAHEPATITIS_B','VITAMINA_A','VITAMINA_K','TAMIZAUDITIVO','PRODUCTOEMBARAZO','TIPOCESAREA','UTILIZOFORCEPS','LUGARNACIMIENTO','CLUES','PERSONALATENDIO','ENTIDADFEDERATIVAPARTO','MUNICIPIOPARTO','LOCALIDADPARTO','CERTIFICADOPOR','ENTIDADFEDERATIVACERTIFICA','MUNICIPIOCERTIFICA','LOCALIDADCERTIFICA','FECHACERTIFICADO']]

In [115]:
natalidad_2020_sonora
natalidad_2020_sonora.to_csv('./datos/Sonora_2020.csv',index=False)

In [116]:
Sonora2020=pd.read_csv('./datos/Sonora_2020.csv')

In [117]:
Sonora2020.dtypes

ENTIDADNACIMIENTO              int64
MUNICIPIONACIMIENTO            int64
FECHANACIMIENTOMADRE          object
EDAD                           int64
SECONSIDERAINDIGENA            int64
HABLALENGUAINDIGENA            int64
ESTADOCONYUGAL                 int64
ENTIDADRESIDENCIA              int64
MUNICIPIORESIDENCIA            int64
LOCALIDADRESIDENCIA            int64
NUMEROEMBARAZOS                int64
HIJOSNACIDOSMUERTOS            int64
HIJOSNACIDOSVIVOS              int64
HIJOSSOBREVIVIENTES            int64
CONDICIONHIJOANTERIOR          int64
VIVEHIJOANTERIOR               int64
ORDENNACIMIENTO                int64
ATENCIONPRENATAL               int64
TRIMESTREPRIMERCONSULTA        int64
TOTALCONSULTAS                 int64
SOBREVIVIOPARTO                int64
AFILIACION                     int64
ESCOLARIDAD                    int64
TRABAJAACTUALMENTE             int64
FECHANACIMIENTO               object
HORANACIMIENTO                object
SEXO                           int64
E